In [1]:
import pandas as pd

In [3]:
df = pd.read_csv(r'C:\Users\alvar\Downloads\customer_churn.csv')

In [4]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [21]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd

In [22]:
# Convertir la columna 'TotalCharges' a numérica, forzando errores a NaN (en caso de que existan)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Rellenar los valores NaN que pudieran haber resultado de la conversión con el valor medio de la columna
df['TotalCharges'].fillna(df['TotalCharges'].mean(), inplace=True)

# Separar las características (X) y la variable objetivo (y)
X = df.drop(columns=['Churn', 'customerID'])  # 'customerID' no es relevante para el modelo
y = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_19552\4186326315.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalCharges'].fillna(df['TotalCharges'].mean(), inplace=True)


In [23]:
# Identificar columnas categóricas
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

In [24]:
# Crear un preprocesador para convertir las columnas categóricas a variables numéricas usando OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)],
    remainder='passthrough'  # Mantiene las columnas numéricas tal como están
)

# Aplicar el preprocesador a los datos de entrenamiento
X_train_processed = preprocessor.fit_transform(X_train)

In [25]:
# Aplicar SMOTE para aumentar los datos de la clase minoritaria
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_processed, y_train)

# Aplicar el preprocesador a los datos de prueba
X_test_processed = preprocessor.transform(X_test)

In [26]:
# Modelo 1: Regresión Logística
logreg = LogisticRegression(random_state=42, max_iter=1000)
logreg.fit(X_train_smote, y_train_smote)
y_pred_logreg = logreg.predict(X_test_processed)
accuracy_logreg = accuracy_score(y_test, y_pred_logreg)
accuracy_logreg

c:\Users\alvar\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7586370089919545

In [27]:
# Modelo 2: Árbol de Decisión
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train_smote, y_train_smote)
y_pred_tree = tree.predict(X_test_processed)
accuracy_tree = accuracy_score(y_test, y_pred_tree)
accuracy_tree

0.7250354945575012

El modelo de Regresión Logística tiene una mayor precisión (75.86%) en comparación con el modelo de Árbol de Decisión (72.50%). Esto sugiere que la Regresión Logística ha sido más efectiva en este caso para predecir correctamente la variable objetivo (Churn), después de haber aplicado SMOTE para equilibrar las clases.